In [ ]:
from cleanup_tagged_resources import get_tagged_resources, delete_resources
import boto3

## Delete resources by tag

In [ ]:
resources = get_tagged_resources("app", "pace_bootcamp")
for r in resources:
    print(f"Found {r}")

In [ ]:
delete_resources(resources)

## Handle deletion of untagged resources

In [ ]:
agentcore_control_client = boto3.client('bedrock-agentcore-control')
ecr_client = boto3.client('ecr')
secrets_client = boto3.client('secretsmanager')
codebuild = boto3.client('codebuild')

### Delete AgentCore Runtimes

In [ ]:
runtimes = agentcore_control_client.list_agent_runtimes()['agentRuntimes']
for runtime in runtimes:
    runtime_id = runtime['agentRuntimeId']
    status = runtime.get('status', 'UNKNOWN')

    if status == 'DELETING':
        print(f"Skipping AgentCore Runtime {runtime_id} - already deleting")
        continue
        
    print(f"Deleting AgentCore Runtime {runtime_id}...")
    agentcore_control_client.delete_agent_runtime(
        agentRuntimeId=runtime_id
    )

### Delete Amazon Elastic Container Registry repositories 

In [ ]:
print("Deleting ECR repository...")
repositories = ecr_client.describe_repositories()['repositories']
for repository in repositories:
    name = repository['repositoryName']
    if name.startswith('bedrock-agentcore-mcp_server_agentcore'):
        ecr_client.delete_repository(
            repositoryName=name,
            force=True
        )
print("✓ ECR repository deleted")

### Delete Secrets Manager Secrets

In [ ]:
# Get list of all secrets
secrets = secrets_client.list_secrets()

# Iterate through secrets and delete those starting with mcp_server
for secret in secrets['SecretList']:
    print(secret)
    # secrets_client.delete_secret(
    #     SecretId=secret['ARN'],
    #     ForceDeleteWithoutRecovery=True
    # )

### Delete Codebuild projects

In [ ]:
projects = codebuild.list_projects()['projects']
for project in projects:
    print(project)
    if project.startswith('bedrock-agentcore-'):
        codebuild.delete_project(name=project)